In [1]:
import pandas as pd
import os
import warnings
os.environ["PYTHONIOENCODING"] = "utf-8"
warnings.filterwarnings(action='ignore')

In [2]:
nsmc_train_df = pd.read_csv('./chapter13 data/ratings_train.txt', encoding='utf8', sep='\t', engine='python')
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [4]:
nsmc_train_df = nsmc_train_df[nsmc_train_df['document'].notnull()]

In [5]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [6]:
nsmc_train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [7]:
import re

In [8]:
nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
nsmc_test_df = pd.read_csv('./chapter13 data/ratings_test.txt', encoding='utf8', sep='\t', engine='python')
nsmc_test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [10]:
nsmc_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [11]:
nsmc_test_df = nsmc_test_df[nsmc_test_df['document'].notnull()]

In [12]:
print(nsmc_test_df['label'].value_counts())

1    25171
0    24826
Name: label, dtype: int64


In [13]:
nsmc_test_df['document'] = nsmc_test_df['document'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))

In [14]:
from konlpy.tag import Okt
JVM_PATH = '/Library/Java/JavaVirtualMachines/jdk1.8.0_351.jdk/Contents/Home/bin/java'

okt = Okt()

In [15]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(nsmc_train_df['document'])
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

In [17]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)

In [18]:
SA_lr.fit(nsmc_train_tfidf, nsmc_train_df['label'])

LogisticRegression(random_state=0)

In [19]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [20]:
SA_lr_grid_cv.fit(nsmc_train_tfidf, nsmc_train_df['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [21]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 3} 0.8553


In [22]:
# 최적 파라미터의 best 모델 저장
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [23]:
# 평가용 데이터의 피처 벡터화 : 실행시간 6분 정도 걸립니다 ☺
nsmc_test_tfidf = tfidf.transform(nsmc_test_df['document'])

In [24]:
test_predict = SA_lr_best.predict(nsmc_test_tfidf)

In [25]:
from sklearn.metrics import accuracy_score

print('감성 분석 정확도 : ', round(accuracy_score(nsmc_test_df['label'], test_predict), 3))

감성 분석 정확도 :  0.857


In [60]:
movie1_df = pd.read_csv('./movie/movie1.csv', encoding='utf8', sep=',', engine='python')
movie2_df = pd.read_csv('./movie/movie2.csv', encoding='utf8', sep=',', engine='python')
movie1_df.head()

,Unnamed: 0,고유번호,제목,내용
0,0,4560389,범죄도시 시사회 다녀와서 느낀 진짜 실화 평 드립니다.,\n범죄도시 시사회 다녀와서 느낀 진짜 실화 평 드립니다.어제 잠실 롯데월드몰 저녁...
1,1,4566947,진짜 재미없음,\r\n\t\t\t\t누가 베테랑급이랬니? 니 그라믄 아이댄다연출이 개떡같은데 마동...
2,2,4564060,범죄도시 : 마동석씨와 윤계상씨가 간신히 떠받치고 있었던 청불 조폭 영화,"\n강윤성 감독 / 121분 / 청소년 관람불가마동석, 윤계상, 조재윤, 최귀화, ..."
3,3,4557114,[범죄도시 VIP 시사회 후기] 진짜 대박대박 !!!,\n범죄도시 시사회 다녀왔어요!솔직히 이거 시사회 가자고했을 땐 별 기대 안했는데....
4,4,4557101,기대이상! 킬링타임용으로 생각했는데 대작에 낚임,\n\r\n마요미와 윤계상 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ아오 포스터랑 예고편이 다 못살림 ㅠ...


In [61]:
movie1_df['combin'] = movie1_df["제목"] + " " +movie1_df["내용"]
movie2_df['combin'] = movie2_df["제목"] + " " +movie2_df["내용"]
movie1_df.head()

,Unnamed: 0,고유번호,제목,내용,combin
0,0,4560389,범죄도시 시사회 다녀와서 느낀 진짜 실화 평 드립니다.,\n범죄도시 시사회 다녀와서 느낀 진짜 실화 평 드립니다.어제 잠실 롯데월드몰 저녁...,범죄도시 시사회 다녀와서 느낀 진짜 실화 평 드립니다. \n범죄도시 시사회 다녀와서...
1,1,4566947,진짜 재미없음,\r\n\t\t\t\t누가 베테랑급이랬니? 니 그라믄 아이댄다연출이 개떡같은데 마동...,진짜 재미없음 \r\n\t\t\t\t누가 베테랑급이랬니? 니 그라믄 아이댄다연출이 ...
2,2,4564060,범죄도시 : 마동석씨와 윤계상씨가 간신히 떠받치고 있었던 청불 조폭 영화,"\n강윤성 감독 / 121분 / 청소년 관람불가마동석, 윤계상, 조재윤, 최귀화, ...",범죄도시 : 마동석씨와 윤계상씨가 간신히 떠받치고 있었던 청불 조폭 영화 \n강윤성...
3,3,4557114,[범죄도시 VIP 시사회 후기] 진짜 대박대박 !!!,\n범죄도시 시사회 다녀왔어요!솔직히 이거 시사회 가자고했을 땐 별 기대 안했는데....,[범죄도시 VIP 시사회 후기] 진짜 대박대박 !!! \n범죄도시 시사회 다녀왔어요...
4,4,4557101,기대이상! 킬링타임용으로 생각했는데 대작에 낚임,\n\r\n마요미와 윤계상 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ아오 포스터랑 예고편이 다 못살림 ㅠ...,기대이상! 킬링타임용으로 생각했는데 대작에 낚임 \n\r\n마요미와 윤계상 ㅋㅋㅋㅋ...


In [62]:
movie1_df_list = movie1_df['combin'].values.tolist()
movie2_df_list = movie2_df['combin'].values.tolist()

In [63]:
good_movie1 = 0
good_movie2 = 0
bad_movie1 = 0
bad_movie2 = 0
for review in movie1_df_list:
    review = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(review)
    review = [" ".join(review)]
    review_tfidf = tfidf.transform(review)
    review_predict = SA_lr_best.predict(review_tfidf)
    if(review_predict== 0):
        bad_movie1 += 1
    else :
        good_movie1 += 1
        
for review in movie2_df_list:
    review = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(review)
    review = [" ".join(review)]
    review_tfidf = tfidf.transform(review)
    review_predict = SA_lr_best.predict(review_tfidf)
    if(review_predict== 0):
        bad_movie2 += 1
    else :
        good_movie2 += 1
print("범죄도시1 : 긍정{0}건, 부정{1}건\n범죄도시2 : 긍정{2}건, 부정{3}건".format(good_movie1, bad_movie1, good_movie2, bad_movie2))

범죄도시1 : 긍정804건, 부정182건
범죄도시2 : 긍정204건, 부정82건
